In [1]:
import os
import json
import sys
import numpy as np
import random
from smol.io import load_work
from pymatgen.core.structure import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.io.vasp.inputs import Poscar
from ase.db import connect
from ase.io import write
from ase.visualize import view

sys.path.append('../Modules')
from structure_generation import * 
from vasp_misc import *


In [2]:
json_file = '../Notebooks/compositions_gen2.json'
output_dir = '../Visualization/Job_Structures/Pre_VASP/VCrTiWZr_Summit/Gen_0-2/Perfect'

# make the output directory
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



In [3]:
supercells = create_supercells_for_compositions(json_file, output_dir, num_structures=3, lattice_parameter=3.01 , supercell_type='prim')

ValueError: Sample larger than population or is negative